# Pipline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import glob

data = pd.read_excel("OMSK-PKN_GP1.1_NEW.xlsx", sheet_name="Фасады_Секции по файлу")
data

#==== Concat all csv
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_OM" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df


#=== Clean data
def clean_start_data(data):
    revit_data = multi_concat()
    df = data[['Unnamed: 2', 'Unnamed: 3']]
    df = df.dropna()

    df = df[df['Unnamed: 2'].str.count(r"\.") > 1]

    df.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
    df = df.reset_index(drop=True)
    df['ElementId'] = df['ElementId'].astype('int64')
    tangle_data = df
    #=== Marge
    df_groups = df.groupby('ElementId')['Position'].apply(list).reset_index()
    marged_df = df_groups.merge(revit_data, left_on='ElementId', right_on='ElementId')

    return marged_df

marged_df = clean_start_data(data)

C:\Users\homer\AppData\Local\Temp\ipykernel_14300\1149295551.py:18: DtypeWarning: Columns (58,83,100,101,106,112,116,119,121,123,124,142,143,149,150,152,156,157,164,167,170,175,177,183,185,186,187,188,189,190,194,196,198,204,207,211,212,215,231,238,239,249,253,286,318,319,323,331,332,333,339,344,345,346,347,348,403,407,408,409,410,432,448,450,480,483,488,489,490,491,492,493,494,495,498,499,500,572,591,592,595,596,600,601,665,673,675,676,678,680,696,697,698,699,700,701,702,703,704,705,706,707,711,712,713,714,715,716,726,727,730,731,732,733,734,735,736,740,741,742,743,744,745,746,747,749,751,759,760,762,768,777,778,779,780,782,809,810,833,837,858,868,869,870,871,872,875,876,887,888,889,890,891,892,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,958,973,974,975,976,983,993,994,995,996,1010,1013,1014,1015,1019,1028,1030,1031,1032,1033,1040,1041,1042,1043,1044,1045

The data is imbalance. <br>
I can remember about it.

In [25]:
#=== marged_df split and clear
def get_data(marged_df):
    target_cols = ['ElementId', 'Position']
    X_df = marged_df.drop(columns=target_cols, axis=1)
    X_df_to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации', 'ADSK_Номер секции','ADSK_Группирование']
    X_df.drop(columns=X_df_to_drop, inplace=True)
    X_df_to_drop2 =  X_df.columns[X_df.columns.str.contains('PS_Секция')]
    X_df.drop(columns=X_df_to_drop2, inplace=True)
    X_df['Зависимость снизу'] = X_df['Зависимость снизу'].str.split('(', expand=True)[0]
    X_df['Зависимость сверху'] = X_df['Зависимость сверху'].str.split('(', expand=True)[0]
    X_df['Уровень'] = X_df['Уровень'].str.split('(', expand=True)[0]

    y_df = marged_df[target_cols]
    y_df_to_drop = []

    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    fit_transform = mlb.fit_transform(y_df['Position'])
    y_df = pd.DataFrame(fit_transform, columns=mlb.classes_,index=y_df['ElementId'])
    y_df = y_df.add_prefix('Target_')
    for col in y_df.columns:
        # print(f'{col}: 0-{y_df[col].value_counts()[0]}, 1-{y_df[col].value_counts()[1]}')
        # print(y_df[col].sum())
        if y_df[col].sum() < 5:
            y_df_to_drop.append(col)
    y_df.drop(columns=y_df_to_drop, inplace=True)

    return X_df,y_df

X_df,y_df = get_data(marged_df = marged_df)
constant_cols = X_df.columns[X_df.nunique() <= 1]
X_df = X_df.drop(columns=constant_cols)
print("Константные колонки:", len(list(constant_cols)))

# X_df.fillna("NAN", inplace=True)

Константные колонки: 2046


In [26]:
X_df['ADSK_Единица измерения'].value_counts()

ADSK_Единица измерения
м²     2
шт.    1
Name: count, dtype: int64

In [30]:
for col in y_df.columns:
    count = y_df[col].value_counts()
    print(f'{col}: 0-{count[0]}, 1-{count[1]}')

Target_004.014.033.001.002: 0-4058, 1-1334
Target_004.014.033.001.005: 0-3301, 1-2091
Target_004.014.033.001.008: 0-3301, 1-2091
Target_004.014.033.001.012: 0-3301, 1-2091
Target_004.014.033.001.013: 0-3301, 1-2091
Target_004.014.033.002.001: 0-3936, 1-1456
Target_004.014.033.002.002: 0-3936, 1-1456
Target_004.014.033.002.003: 0-3936, 1-1456
Target_004.014.033.002.004: 0-3936, 1-1456
Target_004.014.033.002.005: 0-3936, 1-1456
Target_004.014.033.004.001: 0-5157, 1-235
Target_004.014.033.004.002: 0-4843, 1-549
Target_004.014.033.004.003: 0-5157, 1-235
Target_004.014.033.004.004: 0-5157, 1-235
Target_004.014.033.004.005: 0-5157, 1-235
Target_004.014.033.004.006: 0-5157, 1-235
Target_004.014.033.006.001.1: 0-5346, 1-46
Target_021.015.001.017: 0-5163, 1-229
Target_031.003.002.001: 0-5262, 1-130
Target_031.003.002.002: 0-4464, 1-928
Target_031.003.002.003: 0-5377, 1-15
Target_031.006.003: 0-5121, 1-271
Target_031.006.006: 0-5353, 1-39


## Обучение

STk-fold

In [31]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
import joblib
from sklearn.model_selection import train_test_split

import json
cat_features = []
for col in X_df.select_dtypes('object').columns:
    X_df[col] = X_df[col].astype(str)
    cat_features.append(col)
    X_df[col] = X_df[col].astype('category')

catb_model = CatBoostClassifier(
    random_state=42,
    nan_mode='Max',           # или 'Min' тут хз
    max_depth=6,              
    learning_rate=0.1,        # мб стоит изсенить 
    n_estimators=200,         # количество деревьев пока 200
    verbose=100,              # лог каждые 100 итераций, иначе иного инфы
    scale_pos_weight=None,    # комит, так как не задаю class_weights
    cat_features=cat_features,# база
    early_stopping_rounds=30,
    eval_metric='F1',         # попробую задалансить f1
    auto_class_weights='Balanced',  # !автомитическая балансировака классов. Имба
)

multi_catb = []
for i in range(y_df.shape[1]):
    y_i = y_df.iloc[:, i]

    cbc = CatBoostClassifier(**catb_model.get_params())
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_df, 
        y_i, 
        test_size=0.33, 
        random_state=42, 
        stratify=y_i
    )
    
    cbc.fit(X_df, y_i, plot=True,eval_set=(X_val, y_val))

    multi_catb.append(cbc)

joblib.dump(multi_catb, 'catb.pkl')



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9884926	test: 0.9870593	best: 0.9870593 (0)	total: 34.5ms	remaining: 6.87s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9996265789
bestIteration = 25

Shrink model to first 26 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9454389	test: 0.9451502	best: 0.9451502 (0)	total: 50.6ms	remaining: 10.1s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.9942510295
bestIteration = 38

Shrink model to first 39 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9868003	test: 0.9844875	best: 0.9844875 (0)	total: 37ms	remaining: 7.37s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 1
bestIteration = 7

Shrink model to first 8 iterations.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9996265	test: 0.9994399	best: 0.9994399 (0)	total: 38ms	remaining: 7.56s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 1
bestIteration = 2

Shrink model to first 3 iterations.


['catb.pkl']

In [33]:
import joblib
import json

# Сохранение моделей, cat_features и фичей
# joblib.dump(multi_lgbm, 'lgbm.pkl')
joblib.dump(multi_catb, 'catb.pkl')

with open('features.json', 'w') as f:
    json.dump(X_df.columns.to_list(), f)

print("Модели и метаданные сохранены.")

Модели и метаданные сохранены.


In [34]:
with open('trained_targets.json', 'w') as f:
    json.dump(y_df.columns.to_list(), f)

In [35]:
with open('cat_features.json', 'w') as f:
    json.dump(cat_features, f)

In [38]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

#=== Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ensemble = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_catb]).T

# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

#=== отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ensemble > 0.5).astype(int)

In [10]:
y_df.shape, y_pred_ensemble.shape

((5392, 23), (5392, 23))

In [39]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score

#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(y_df, y_pred_ensemble, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(y_df, y_pred_ensemble)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(y_df, y_pred_ensemble, average='macro')
f1_micro = f1_score(y_df, y_pred_ensemble, average='micro')

precision_macro = precision_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
precision_micro = precision_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

recall_macro = recall_score(y_df, y_pred_ensemble, average='macro', zero_division=0)
recall_micro = recall_score(y_df, y_pred_ensemble, average='micro', zero_division=0)

#=== Метрики
print("Результаты ансамбля LGBM + CatBoost:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost:
Macro F1 Score:       0.9776
Micro F1 Score:       0.9948
Macro Precision:      0.9640
Micro Precision:      0.9931
Macro Recall:         0.9975
Micro Recall:         0.9964


# Проверка на тесовых данных

Интерпретация результатов 

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import MultiLabelBinarizer

test_data = pd.read_excel("KRGN-MK_GP07_Фасады_NEW.xlsx", sheet_name="Фасады_Секции по файлу")

#=== Clean data
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    
    for file in csv_files:
        if "revit_params_KR" in file:
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df
test_marged_df = clean_start_data(test_data)

target_cols = ['ElementId', 'Position']
X = test_marged_df.drop(columns=target_cols, axis=1)
y = test_marged_df[target_cols]

with open('features.json', 'r') as f:
    features = json.load(f)
    
columns_to_drop = set(X.columns) - set(features)
X = X.drop(columns=columns_to_drop)
X

C:\Users\homer\AppData\Local\Temp\ipykernel_14300\1270042712.py:17: DtypeWarning: Columns (158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,186,189,195,208,220,221,225,226,227,231,234,235,237,240,245,247,253,254,257,258,259,260,261,265,267,268,274,277,281,283,284,286,303,309,310,313,314,315,328,329,358,364,396,397,402,411,414,415,416,423,424,425,426,427,428,454,457,467,469,470,475,476,500,517,554,555,556,557,558,559,560,561,639,647,648,649,662,663,666,667,668,669,671,672,755,756,757,784,785,786,787,788,789,796,801,802,804,805,806,807,808,809,812,813,814,815,816,817,818,819,820,821,822,824,826,827,845,847,849,879,880,898,902,948,951,952,953,954,961,966,967,968,969,970,971,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1020,1034,1036,1048,1052,1053,1054,1055,1056,1057,1058,1064,1075,1092,1093,109

,ADSK_Единица измерения,ADSK_Зона,ADSK_Количество,ADSK_Материал,ADSK_Материал тип подсчета,ADSK_Наименование,ADSK_Норма расхода,ADSK_Обозначение,ADSK_Основной материал элемента,ADSK_Примечание,...,яВырезНизСредний,яВырезПравыйВерх,яВырезПравыйСредний,#_Высота отделки,#_Ширина откоса,Длина выступа_Откоса_Верх,Длина выступа_Откоса_Слева,Длина выступа_Откоса_Справа,Длина выступа_Откоса_Низ,Материал_Отделки
0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11015,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11016,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11017,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11018,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
columns_empty = set(features) - set(X.columns)
if columns_empty:
    for column in columns_empty:
        X[column] = 0

In [15]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import json

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)


catb_model = CatBoostClassifier(
    random_state=42,
    nan_mode='Max',           # или 'Min' тут хз
    max_depth=6,              
    learning_rate=0.1,        # мб стоит изсенить 
    n_estimators=200,         # количество деревьев пока 200
    verbose=100,              # лог каждые 100 итераций, иначе иного инфы
    scale_pos_weight=None,    # комит, так как не задаю class_weights
    cat_features=test_cat_features,# база
    early_stopping_rounds=30,
    eval_metric='F1',         # попробую задалансить f1
    auto_class_weights='Balanced',  # !автомитическая балансировака классов. Имба
)


#=== Загрузка
print("Загрузка CatBoost завершена")
multi_catb = joblib.load('catb.pkl')

Загрузка CatBoost завершена


In [27]:
test_cat_features

['ADSK_Единица измерения',
 'ADSK_Зона',
 'ADSK_Материал',
 'ADSK_Наименование',
 'ADSK_Обозначение',
 'ADSK_Основной материал элемента',
 'ADSK_Примечание',
 'Category',
 'Группа модели',
 'Зависимость сверху',
 'Зависимость снизу',
 'Категория',
 'Материал несущих конструкций',
 'Материал отделки основной',
 'Материал откоса верхний',
 'Материал откоса левый',
 'Материал откоса нижний',
 'Материал откоса правый',
 'Материал_Отделки',
 'Материал_Отделки_Верх',
 'Материал_Откоса',
 'Материал_Откоса_Верх',
 'Материал_Откоса_Низ',
 'Материал_Откоса_Слева',
 'Материал_Откоса_Справа',
 'Материал_Утеплитель_Верх',
 'Описание',
 'Основа',
 'Рабочий набор',
 'Семейство',
 'Семейство и типоразмер',
 'Стадия возведения',
 'Стадия сноса',
 'Тип',
 'Уровень',
 'Штриховка при низкой детализации']

In [24]:
X['ADSK_Единица измерения'].value_counts()

ADSK_Единица измерения
м.п.    621
м²       15
шт.       4
шт        3
м³        3
м.п       2
Name: count, dtype: int64

In [17]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

#=== Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ensemble = np.array([clf.predict_proba(X)[:, 1] for clf in multi_catb]).T

# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

#=== отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ensemble > 0.5).astype(int)

CatBoostError: Bad value for num_feature[non_default_doc_idx=2018,feature_idx=0]="шт": Cannot convert 'шт' to float

Логика в том что берем только те таргеты которые были в обучении

In [15]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. y_pred_ensemble.shape - (220244, 29)
Надо занулить лишние таргеты, что бы занулить то, что не предсказано, что бы не ломать метрики. но
У меня сейчас супер мало данных. Так что придется проверять только то что есть. 
"""
y.columns.__len__() # - 37 таргеты на тестовых данных 
y_df.columns.__len__() # - 25 таргеты на обучающих данных 

expected_targets = y.columns.tolist()
with open('trained_targets.json', 'r') as f:
    trained_targets = json.load(f)
trained_targets
print(expected_targets)
print(trained_targets)

indexes = []
data_targets = []
for ind, targ in enumerate(trained_targets):
    if targ in expected_targets :
        print(targ)
        indexes.append(ind)
        data_targets.append(targ)

new_pred = y_pred_ensemble[:, indexes]
new_y = y[data_targets]
# new_y
new_y

['Target_004.014.033.001.002', 'Target_004.014.033.001.005', 'Target_004.014.033.001.008', 'Target_004.014.033.001.012', 'Target_004.014.033.001.013', 'Target_004.014.033.001.015', 'Target_004.014.033.001.3', 'Target_004.014.033.002.001', 'Target_004.014.033.002.002', 'Target_004.014.033.002.003', 'Target_004.014.033.002.004', 'Target_004.014.033.002.005', 'Target_004.014.033.004.001', 'Target_004.014.033.004.002', 'Target_004.014.033.004.003', 'Target_004.014.033.004.004', 'Target_004.014.033.004.005', 'Target_004.014.033.004.006', 'Target_004.014.033.006.001.1', 'Target_004.014.033.006.002', 'Target_021.004.003', 'Target_021.005.003.005.073', 'Target_021.005.003.005.074', 'Target_021.005.003.005.075', 'Target_021.005.003.005.082', 'Target_021.015.001.017', 'Target_031.003.002.006', 'Target_031.003.003.001', 'Target_031.003.003.002', 'Target_031.003.003.003', 'Target_031.003.003.004', 'Target_031.003.003.005', 'Target_031.006.003', 'Target_031.006.004', 'Target_031.006.006', 'Target_0

,Target_004.014.033.001.002,Target_004.014.033.001.005,Target_004.014.033.001.008,Target_004.014.033.001.012,Target_004.014.033.001.013,Target_004.014.033.002.001,Target_004.014.033.002.002,Target_004.014.033.002.003,Target_004.014.033.002.004,Target_004.014.033.002.005,Target_004.014.033.004.001,Target_004.014.033.004.002,Target_004.014.033.004.003,Target_004.014.033.004.004,Target_004.014.033.004.005,Target_004.014.033.004.006,Target_004.014.033.006.001.1,Target_021.015.001.017,Target_031.006.003,Target_031.006.006
ElementId,,,,,,,,,,,,,,,,,,,,
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128761,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5309498,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6114827,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
6116494,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
# тут сравнения по 2-м датафреймам
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
#=== Точность по объектам 
exact_match = accuracy_score(new_y,new_pred, normalize=True)

#=== Hamming Accuracy - доля правильно предсказанных меток
hamming_acc = 1 - hamming_loss(new_y, new_pred)

#=== F1, Precision, Recal
f1_macro = f1_score(new_y, new_pred, average='macro')
f1_micro = f1_score(new_y, new_pred, average='micro')

precision_macro = precision_score(new_y, new_pred, average='macro', zero_division=0)
precision_micro = precision_score(new_y, new_pred, average='micro', zero_division=0)

recall_macro = recall_score(new_y, new_pred, average='macro', zero_division=0)
recall_micro = recall_score(new_y, new_pred, average='micro', zero_division=0)

#=== Метрики
print("Результаты CatBoost на тестовых данных:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты CatBoost на тестовых данных:
Macro F1 Score:       0.5106
Micro F1 Score:       0.7218
Macro Precision:      0.5085
Micro Precision:      0.7061
Macro Recall:         0.5720
Micro Recall:         0.7381
